# Deep Research


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.sandbox.google.com/github/hupili/google-adk-101/blob/main/Deploy_to_Agent_Engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fhupili%2Fgoogle-adk-101%2Fmain%2FDeploy_to_Agent_Engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-enterprise-logo.png" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/hupili/google-adk-101/blob/main/Deploy_to_Agent_Engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/monospace/240802/git_host_github_mask.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/hupili/google-adk-101/blob/main/Deploy_to_Agent_Engine.ipynb">
      <img width="32px" src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
| Author(s) | [Pili Hu](https://github.com/hupili)|
| Reviewers(s) | Nil |
| Last updated | 2025-06-02 |
| google-adk | 1.1.1 |
| vertexai | 1.93.1 |

## Environment

In [1]:
import os

# API_KEY = '' # @param {type:"string"}
# os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE" # Use Vertex AI API
# os.environ["GOOGLE_API_KEY"] = API_KEY

PROJECT_ID = "hupili-genai-bb"  # @param {type:"string"}
if not PROJECT_ID:
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "us-central1" # @param {type:"string"}

STAGING_BUCKET = 'gs://agent_engine_deploy_staging' # @param {type:"string"}

os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE" # Use Vertex AI API

# [your-project-id]

In [2]:

import vertexai

vertexai.init(
    project=PROJECT_ID,
    location=LOCATION,
    staging_bucket=STAGING_BUCKET,
)

In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
!pip install google-adk==1.1.1

In [5]:
from google import adk

In [6]:
adk.__version__

'1.1.1'

In [7]:
vertexai.__version__

'1.93.1'

## Util functions

In [13]:
from vertexai import agent_engines
from vertexai.preview import reasoning_engines

In [9]:
def delete(resource_id: str) -> None:
    remote_agent = agent_engines.get(resource_id)
    remote_agent.delete(force=True)
    print(f"Deleted remote agent: {resource_id}")

def list_agents() -> None:
    remote_agents = agent_engines.list()
    TEMPLATE = '''
{agent.name} ("{agent.display_name}")
- Create time: {agent.create_time}
- Update time: {agent.update_time}
'''
    remote_agents_string = '\n'.join(TEMPLATE.format(agent=agent) for agent in remote_agents)
    print(f"All remote agents:\n{remote_agents_string}")

In [10]:
list_agents()

All remote agents:

5147799092037943296 ("")
- Create time: 2025-06-02 14:14:28.600149+00:00
- Update time: 2025-06-02 14:18:55.955685+00:00


2164164338904989696 ("")
- Create time: 2025-06-02 11:57:47.050939+00:00
- Update time: 2025-06-02 12:01:18.059991+00:00


6087608854902734848 ("DICE_AGENT")
- Create time: 2025-04-25 08:19:20.817490+00:00
- Update time: 2025-04-25 08:22:05.936293+00:00


7916070303615156224 ("llm_auditor")
- Create time: 2025-04-25 07:13:25.339138+00:00
- Update time: 2025-04-25 07:18:12.898631+00:00


5307078744484085760 ("llm_auditor")
- Create time: 2025-04-09 08:22:36.179879+00:00
- Update time: 2025-04-09 08:25:42.734584+00:00


8402318325882814464 ("")
- Create time: 2025-03-28 08:42:23.121480+00:00
- Update time: 2025-03-28 08:46:29.629436+00:00


66410502317670400 ("my-awesome-agent")
- Create time: 2025-03-07 01:22:05.930759+00:00
- Update time: 2025-03-07 01:33:02.201959+00:00



In [11]:
def call_remote_agent_engine_app(app, user_id, session_id, query):
  response_text = ''
  for event in app.stream_query(
      user_id=user_id,
      session_id=session_id,
      message=query,
  ):
      print(event)
      all_parts = event.get('content', {}).get('parts', [])
      for part in all_parts:
          response_text += (part.get('text', ''))
  return response_text

## Simple Search Agent

In [14]:
from google.adk.tools import google_search
from google.adk import Agent
from google.adk.agents import Agent, LlmAgent
from google.genai import types
from pydantic import BaseModel
from google.genai import types
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

MODEL = "gemini-2.0-flash"
APP_NAME = "simple_search_example"
USER_ID = "user12345"
SESSION_ID = "session12345"
AGENT_NAME = "simple_search_agent"

# Agent
simple_search_agent = Agent(
    model=MODEL,
    name="simple_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

app = reasoning_engines.AdkApp(
    agent=simple_search_agent,
    enable_tracing=True,
)

In [15]:
session = await app.create_session(user_id=USER_ID)
session

Session(id='84fb74fa-bf03-45e5-b802-88f753580004', app_name='default-app-name', user_id='user12345', state={}, events=[], last_update_time=1748874587.3218415)

In [16]:
call_remote_agent_engine_app(app, USER_ID, session.id, "what is the weather in New York")

{'content': {'parts': [{'text': 'The weather in New York City, NY is currently sunny with a temperature of 66°F (19°C). It feels like 70°F (21°C). The humidity is around 43%.\n\nThe forecast for today, June 2, 2025, is partly cloudy during the day and clear at night. There is a 10% chance of rain during the day and a 15% chance at night. The temperature will be between 56°F (13°C) and 72°F (22°C).\n'}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'google.com', 'title': 'Weather information for locality: New York, administrative_area: NY', 'uri': 'https://www.google.com/search?q=weather+in+New York,+NY'}}], 'grounding_supports': [{'confidence_scores': [0.7873515], 'grounding_chunk_indices': [0], 'segment': {'end_index': 236, 'start_index': 147, 'text': 'The forecast for today, June 2, 2025, is partly cloudy during the day and clear at night.'}}, {'confidence_scores': [0.82351136], 'grounding_chunk_indices': [0], 'segment': {'end_index': 308, 'start_i

'The weather in New York City, NY is currently sunny with a temperature of 66°F (19°C). It feels like 70°F (21°C). The humidity is around 43%.\n\nThe forecast for today, June 2, 2025, is partly cloudy during the day and clear at night. There is a 10% chance of rain during the day and a 15% chance at night. The temperature will be between 56°F (13°C) and 72°F (22°C).\n'

In [17]:
call_remote_agent_engine_app(app, USER_ID, session.id, "is earth square?")

{'content': {'parts': [{'text': "No, the Earth is not square. While it appears round from space, its shape is more accurately described as an irregular ellipsoid, or oblate spheroid. This means it's like a sphere that is slightly flattened at the poles and bulging at the equator due to the centrifugal force of its rotation.\n"}], 'role': 'model'}, 'grounding_metadata': {'grounding_chunks': [{'web': {'domain': 'wikipedia.org', 'title': 'wikipedia.org', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEZFTPlbIl4Y1lw8Kxz-ClTYPXBNRpk3OMdnhnPTMywNuDNDZ8KR2gDGdk2hbsySwHysGt7puy8DtzRqpcNa1cDOIjbaJf_WCwpwlJPaEO6yngbcRe-XJEFPHcMvCn9OqZYyxs='}}, {'web': {'domain': 'quora.com', 'title': 'quora.com', 'uri': 'https://vertexaisearch.cloud.google.com/grounding-api-redirect/AbF9wXEGkHBOrIxodevtxmD5ABp3O2VeJoFAaXi4As-wOBQGDD_P4_eizIy4em5SyjFTHvjV6ohNkRAY8dfFnOpL-z0hY50buvCfreWDqJIJUKE_DV0dga4PrAWSMzyEZvXsHIOHWyNdk0bKpgP47_hKFpovBawtHTX414W8qMBMGEzFfCIE39Ry3e9quALJnIZ0iU7STRc

"No, the Earth is not square. While it appears round from space, its shape is more accurately described as an irregular ellipsoid, or oblate spheroid. This means it's like a sphere that is slightly flattened at the poles and bulging at the equator due to the centrifugal force of its rotation.\n"

## Search multiple questions


In [19]:
from google.adk.tools.agent_tool import AgentTool

# Instantiate again because each agent tree shall have unique objects.
simple_search_agent = Agent(
    model=MODEL,
    name="simple_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

multiple_search_agent = Agent(
    model=MODEL,
    name="research_agent",
    description="Get answers to multiple questions one by one.",
    instruction="Observe the pending questions in the chat log and call the simple_search_agent to conduct search and give answers to each question. Present the search result in a markdown, one section per question and answer",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[AgentTool(simple_search_agent)],
)

app = reasoning_engines.AdkApp(
    agent=multiple_search_agent,
    enable_tracing=True,
)

session = await app.create_session(user_id=USER_ID)

call_remote_agent_engine_app(app, USER_ID, session.id, """
- Is Earth square?
- Is Moon flat?
- Is Sun cold?
""")

{'content': {'parts': [{'function_call': {'id': 'adk-86084634-fdb5-45ff-be8f-ef3fc5866a35', 'args': {'request': 'Is Earth square?'}, 'name': 'simple_search_agent'}}, {'function_call': {'id': 'adk-482d0901-8ff9-4f48-bf2c-03539b6dc35a', 'args': {'request': 'Is Moon flat?'}, 'name': 'simple_search_agent'}}, {'function_call': {'id': 'adk-a8b821f0-b93c-470e-bf41-fa89e7be6d9c', 'args': {'request': 'Is Sun cold?'}, 'name': 'simple_search_agent'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 30, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 30}], 'prompt_token_count': 107, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 107}], 'total_token_count': 137, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-cfc453fb-eff3-4ac1-80e8-9f3962b47308', 'author': 'research_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'long_running_tool_ids'

{'content': {'parts': [{'function_response': {'id': 'adk-86084634-fdb5-45ff-be8f-ef3fc5866a35', 'name': 'simple_search_agent', 'response': {'result': 'No, the Earth is not square. While it appears round from space, its shape is more accurately described as an irregularly shaped ellipsoid.\n'}}}, {'function_response': {'id': 'adk-482d0901-8ff9-4f48-bf2c-03539b6dc35a', 'name': 'simple_search_agent', 'response': {'result': "Although the moon may appear to be flat to the naked eye, it is actually a sphere. Here's why:\n\n*   **Optical Illusion:** When you look at the moon, it can be difficult to see its 3-dimensional shape due to its distance and the lack of visual cues.\n*   **Telescopic Views:** When viewed through a telescope, the moon appears more textured and round.\n*   **Lunar Phases:** The phases of the moon demonstrate that it is a sphere, as we see different amounts of the illuminated surface as it orbits the Earth.\n*   **Historical Theories:** There was an ancient theory that t

{'content': {'parts': [{'text': '### Is Earth square?\n\nNo, the Earth is not square. While it appears round from space, its shape is more accurately described as an irregularly shaped ellipsoid.\n\n### Is Moon flat?\n\nAlthough the moon may appear to be flat to the naked eye, it is actually a sphere.\n\n### Is Sun cold?\n\nWhile it might seem counterintuitive, the Sun is not cold. The surface of the sun remains at an extremely hot temperature of about 5800 Kelvin all year round.\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 103, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 103}], 'prompt_token_count': 526, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 526}], 'total_token_count': 629, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-cfc453fb-eff3-4ac1-80e8-9f3962b47308', 'author': 'research_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'reques

'### Is Earth square?\n\nNo, the Earth is not square. While it appears round from space, its shape is more accurately described as an irregularly shaped ellipsoid.\n\n### Is Moon flat?\n\nAlthough the moon may appear to be flat to the naked eye, it is actually a sphere.\n\n### Is Sun cold?\n\nWhile it might seem counterintuitive, the Sun is not cold. The surface of the sun remains at an extremely hot temperature of about 5800 Kelvin all year round.\n'

In [23]:
# One sample data related to function call
# data = {
#     'content': {
#         'parts': [
#             {'function_call':
#              {'id': 'adk-86084634-fdb5-45ff-be8f-ef3fc5866a35',
#               'args': {'request': 'Is Earth square?'},
#               'name': 'simple_search_agent'}},
#             {'function_call':
#              {'id': 'adk-482d0901-8ff9-4f48-bf2c-03539b6dc35a',
#               'args': {'request': 'Is Moon flat?'}, 'name': 'simple_search_agent'}},
#             {'function_call':
#              {'id': 'adk-a8b821f0-b93c-470e-bf41-fa89e7be6d9c',
#               'args': {'request': 'Is Sun cold?'}, 'name': 'simple_search_agent'}}

## Generate the research plan

In [24]:
DEEP_RESEARCH_PLAN_INSTRUCTION = """
Instruction for a Deep Research Plan Agent
As a Deep Research Plan Agent, your primary objective is to generate a comprehensive and insightful research plan. This plan will be presented as a markdown bullet point list of questions. The goal is to not only address the user's explicit query but also to anticipate and explore related avenues of inquiry that would be of interest for a thorough understanding of the topic. Your output should guide the user through a structured and deep exploration of their subject.

Here is the process to follow:

Deconstruct the Core Topic:

What is the fundamental subject of the user's request?
What are the primary keywords and concepts?
What is the implicit goal of the user's research? (e.g., to understand a concept, to make a decision, to compare and contrast, to solve a problem).
Generate Foundational Questions:

Start with the basics to establish a strong foundation.
What is the definition of the core topic?
What is its history and origin?
What are the key principles and theories that underpin it?
Who are the seminal thinkers, researchers, or figures associated with this topic?
Explore Deeper, More Nuanced Aspects:

Move beyond the surface-level inquiries.
What are the current debates or controversies surrounding this topic?
What are the different schools of thought or perspectives?
What are the key components, mechanisms, or processes involved? How do they interact?
What are the less obvious or often overlooked aspects of this topic?
Investigate Connections and Relationships:

Think about how the topic connects to broader fields and concepts.
How does this topic relate to other relevant disciplines or areas of study?
What are the upstream causes and downstream effects of the subject?
How has this topic influenced, or been influenced by, historical, social, technological, or economic factors?
Consider Practical Applications and Implications:

Explore the real-world significance of the topic.
What are the practical applications or use cases?
What are the ethical, social, or economic implications?
Who are the key stakeholders affected by or involved in this topic?
Look to the Future:

Encourage forward-thinking inquiry.
What are the emerging trends and future directions of research in this area?
What are the unanswered questions or existing gaps in knowledge?
What are the potential future impacts or developments related to this topic?
Synthesize and Structure the Plan:

Organize the generated questions into a logical flow. A good structure might be:
Foundational Understanding
In-Depth Analysis
Context and Connections
Practical Implications
Future Outlook
Present the final output as a clean, easy-to-read markdown bullet point list.
Use sub-bullets to group related questions and create a hierarchical structure.
Ensure the questions are open-ended and encourage critical thinking rather than simple fact-finding.
By following these instructions, you will create a research plan that is not only deep and thoughtful but also anticipates the user's latent interests, guiding them toward a truly comprehensive understanding of their chosen topic.
"""

In [26]:
# Instantiate again because each agent tree shall have unique objects.
plan_agent = Agent(
    model=MODEL,
    name="plan_agent",
    description="Agent to generate a research plan based on the question or topic given by the user",
    instruction=DEEP_RESEARCH_PLAN_INSTRUCTION,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
)

In [31]:
app = reasoning_engines.AdkApp(
    agent=plan_agent,
    enable_tracing=True,
)

session = await app.create_session(user_id=USER_ID)

response_text = call_remote_agent_engine_app(app, USER_ID, session.id, """AI Agent's impact on labour market""")

{'content': {'parts': [{'text': 'Okay, I can create a research plan to deeply investigate the impact of AI Agents on the labor market. Here\'s a possible plan, structured as a series of questions to guide a comprehensive exploration:\n\n*   **Foundational Understanding**\n    *   What is the definition of an "AI Agent" in the context of the labor market?\n        *   What are some examples of AI Agents currently used in different industries?\n        *   How do AI Agents differ from other forms of automation?\n    *   What are the historical trends of automation and technological advancements impacting the labor market?\n        *   What lessons can be learned from previous industrial revolutions?\n        *   How does the current wave of AI-driven automation compare to previous waves?\n    *   What are the fundamental economic theories related to automation and employment?\n        *   What is the "lump of labor" fallacy and why is it considered a fallacy?\n        *   How do concepts

In [32]:
# prompt: display response_text as markdown

from IPython.display import display, Markdown

display(Markdown(response_text))

Okay, I can create a research plan to deeply investigate the impact of AI Agents on the labor market. Here's a possible plan, structured as a series of questions to guide a comprehensive exploration:

*   **Foundational Understanding**
    *   What is the definition of an "AI Agent" in the context of the labor market?
        *   What are some examples of AI Agents currently used in different industries?
        *   How do AI Agents differ from other forms of automation?
    *   What are the historical trends of automation and technological advancements impacting the labor market?
        *   What lessons can be learned from previous industrial revolutions?
        *   How does the current wave of AI-driven automation compare to previous waves?
    *   What are the fundamental economic theories related to automation and employment?
        *   What is the "lump of labor" fallacy and why is it considered a fallacy?
        *   How do concepts like "skill-biased technological change" and "routine-biased technological change" apply?

*   **In-Depth Analysis: Job Displacement and Creation**
    *   Which specific jobs and industries are most vulnerable to displacement by AI Agents?
        *   What are the characteristics of jobs that are easily automated by AI?
        *   How are wages and employment levels being affected in these vulnerable sectors?
    *   What new jobs and industries are emerging as a result of AI Agent development and deployment?
        *   What skills and training are required for these new roles?
        *   Are these new jobs being created at a rate sufficient to offset job displacement?
    *   How is the nature of work changing due to AI Agents?
        *   Is there a shift towards more "cognitive" or "creative" tasks and away from "routine" tasks?
        *   How are AI Agents augmenting human capabilities in the workplace?
    *   What are the potential impacts on different skill levels and educational backgrounds?
        *   Are low-skilled workers disproportionately affected by AI-driven automation?
        *   How can education and training programs adapt to prepare workers for the future of work?

*   **Context and Connections: Broader Economic and Social Implications**
    *   How is the adoption of AI Agents affecting income inequality?
        *   Is automation contributing to a widening gap between high-skilled and low-skilled workers?
        *   What policy interventions can mitigate the negative impacts on income distribution?
    *   What are the potential impacts on productivity and economic growth?
        *   How can businesses and governments maximize the benefits of AI-driven productivity gains?
        *   Are there risks of "technological unemployment" where productivity gains don't translate into widespread prosperity?
    *   How are different countries and regions being affected differently by AI adoption?
        *   What are the factors that contribute to these differences (e.g., education systems, labor laws, industrial structure)?
        *   What policies are being implemented in different countries to address the challenges and opportunities of AI in the labor market?
    *   What are the ethical considerations surrounding the use of AI Agents in the workplace?
        *   How can we ensure fairness, transparency, and accountability in AI-driven decision-making?
        *   What are the potential biases embedded in AI algorithms and how can they be addressed?

*   **Practical Implications: Strategies for Adaptation and Mitigation**
    *   What strategies can businesses adopt to successfully integrate AI Agents into their workforce?
        *   How can they reskill and upskill their existing employees?
        *   What organizational changes are needed to leverage the full potential of AI?
    *   What role should governments play in preparing the workforce for the age of AI?
        *   What investments are needed in education, training, and infrastructure?
        *   Should governments consider policies like universal basic income or other forms of social safety net?
    *   How can workers protect themselves from job displacement and enhance their career prospects in the age of AI?
        *   What skills and knowledge are most valuable in the AI-driven economy?
        *   How can workers adapt to continuous learning and reskilling throughout their careers?

*   **Future Outlook: Emerging Trends and Unanswered Questions**
    *   What are the emerging trends in AI Agent technology and how might they impact the labor market in the future?
        *   What are the potential breakthroughs in AI that could significantly accelerate automation?
        *   How might the increasing sophistication of AI Agents affect jobs that are currently considered safe from automation?
    *   What are the key unanswered questions and areas for future research?
        *   How can we better predict the long-term impacts of AI on employment and wages?
        *   What are the best strategies for fostering innovation and ensuring that the benefits of AI are widely shared?
    *   What are the potential long-term societal implications of widespread AI adoption in the labor market?
        *   How might it affect social cohesion, political stability, and the overall quality of life?
        *   What are the possible scenarios for the future of work and how can we prepare for them?


## Puting it all together

In [37]:
# Instantiate again because each agent tree shall have unique objects.
plan_agent = Agent(
    model=MODEL,
    name="plan_agent",
    description="Agent to generate a research plan based on the question or topic given by the user",
    instruction=DEEP_RESEARCH_PLAN_INSTRUCTION,
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
)

simple_search_agent = Agent(
    model=MODEL,
    name="simple_search_agent",
    description="Agent to answer questions using Google Search.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[google_search],
)

multiple_search_agent = Agent(
    model=MODEL,
    name="research_agent",
    description="Get answers to multiple questions one by one.",
    instruction="Observe the pending questions in the chat log and call the simple_search_agent to conduct search and give answers to each question. Present the search result in a markdown, one section per question and answer",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
    tools=[AgentTool(simple_search_agent)],
)

report_agent = Agent(
    model=MODEL,
    name="report_agent",
    description="Agent to generate a markdown research report from the questions and their search results from the Internet",
    instruction="Summarize the research results and present research report in markdown format",
    generate_content_config=types.GenerateContentConfig(
        max_output_tokens=8000,
    ),
)

from google.adk.agents.sequential_agent import SequentialAgent

root_agent = SequentialAgent(
    name="deep_researcher",
    description="This is a deep research that makes research plan, use Google search to gather information and generate a report.",
    sub_agents=[plan_agent, multiple_search_agent, report_agent],
)

app = reasoning_engines.AdkApp(
    agent=root_agent,
    enable_tracing=True,
)

In [39]:
session = await app.create_session(user_id=USER_ID)

response_text = call_remote_agent_engine_app(app, USER_ID, session.id, """AI Agent's impact on labour market""")

{'content': {'parts': [{'text': "Okay, I can help you create a research plan to deeply explore the impact of AI Agents on the labor market. Here's a structured plan with questions to guide your research:\n\n*   **Foundational Understanding**\n    *   What is the definition of an AI Agent in the context of the labor market?\n    *   What are the different types of AI Agents currently being used or developed (e.g., chatbots, robotic process automation, virtual assistants)?\n    *   What are the historical trends of automation and technological advancements and their impact on employment?\n    *   What are the key economic theories related to technological unemployment and job creation?\n\n*   **In-Depth Analysis**\n    *   How are AI Agents currently being used in different industries and sectors?\n    *   What specific tasks and roles are most susceptible to automation by AI Agents?\n    *   What are the arguments for and against the displacement of human workers by AI Agents?\n    *   

{'content': {'parts': [{'text': 'Okay, I will use the search tool to answer the questions in your research plan regarding the impact of AI Agents on the labor market. Here are the search results, organized by question:\n\n**Foundational Understanding**\n\n*   **What is the definition of an AI Agent in the context of the labor market?**\n'}, {'function_call': {'id': 'adk-7c0110ae-6cd7-4cbd-9a34-634352666ca1', 'args': {'request': 'definition of AI Agent in labor market context'}, 'name': 'simple_search_agent'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 79, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 79}], 'prompt_token_count': 964, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 964}], 'total_token_count': 1043, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-fdb59cf2-4f91-4c88-8c07-dbdb25cd514c', 'author': 'research_agent', 'actions': {'state_delta': {}, 'art

{'content': {'parts': [{'function_response': {'id': 'adk-7c0110ae-6cd7-4cbd-9a34-634352666ca1', 'name': 'simple_search_agent', 'response': {'result': 'In the context of the labor market, an AI agent can be defined as a sophisticated software program that:\n\n*   **Autonomously performs tasks:** It can work independently with minimal human oversight.\n*   **Perceives its environment:** It gathers information from various sources.\n*   **Makes decisions:** It analyzes data and chooses actions to achieve specific goals.\n*   **Takes action:** It executes tasks and adapts in real-time.\n*   **Learns and improves:** It uses machine learning to optimize its performance over time.\n\nAI agents in the labor market are revolutionizing various industries by:\n\n*   **Automating repetitive tasks:** This allows human employees to focus on more creative, strategic, and innovative work.\n*   **Boosting productivity:** AI agents can process data and complete tasks faster and more accurately than huma

{'content': {'parts': [{'text': 'An AI agent in the labor market is a software program that autonomously performs tasks, perceives its environment, makes decisions, takes action, and learns/improves over time. They automate tasks, boost productivity, create new job roles, improve decision-making, and reshape the workplace.\n\n*   **What are the different types of AI Agents currently being used or developed (e.g., chatbots, robotic process automation, virtual assistants)?**\n'}, {'function_call': {'id': 'adk-376d4aa5-eae5-4f75-8d9c-c6be200be9e0', 'args': {'request': 'types of AI Agents in use in labor market'}, 'name': 'simple_search_agent'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 106, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 106}], 'prompt_token_count': 1352, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 1352}], 'total_token_count': 1458, 'traffic_type': <TrafficType.ON_DEMAND

{'content': {'parts': [{'function_response': {'id': 'adk-376d4aa5-eae5-4f75-8d9c-c6be200be9e0', 'name': 'simple_search_agent', 'response': {'result': 'AI agents are rapidly changing the labor market, automating tasks and creating new opportunities. Here\'s a breakdown of the types of AI agents and their impact:\n\n**1. Types of AI Agents:**\n\n*   **Simple Reflex Agents:** These are the most basic, reacting to environmental conditions based on pre-defined rules. They don\'t consider past experiences. A thermostat is an example (adjusting temperature based on readings).\n*   **Model-Based Reflex Agents:** These agents use an internal model of the environment, tracking changes to make better decisions.\n*   **Goal-Based Agents:** They make decisions based on achieving specific goals, considering multiple options.\n*   **Utility-Based Agents:** These agents consider both goals and the "utility" or desirability of different outcomes.\n*   **Learning Agents:** These agents can adapt and imp

...

{'content': {'parts': [{'function_response': {'id': 'adk-47d943b9-6826-4644-90e8-f3708d38bf27', 'name': 'simple_search_agent', 'response': {'result': 'The historical trends of automation and technological advancements have had a multifaceted impact on employment, characterized by both job displacement and job creation. Here\'s a breakdown of these trends:\n\n**1. Job Displacement:**\n\n*   **Automation replaces routine tasks:** Automation has historically led to the displacement of workers in routine-based jobs, whether physical or cognitive. Examples include the decline of complex craftsman occupations in manufacturing and routine clerical jobs in office work due to electrification and mechanical switching technologies.\n*   **Digital automation and inequality:** Digital automation since the 1980s has contributed to labor market inequality, with production and clerical workers experiencing job losses or wage decline.\n*   **The "new automation" and future displacement:** The "new auto

{'content': {'parts': [{'text': 'Historically, automation has led to both job displacement (especially in routine tasks) and job creation (new jobs requiring specialized skills). It has also contributed to wage stagnation/decline for some workers and increased labor market inequality.\n\n*   **What are the key economic theories related to technological unemployment and job creation?**\n'}, {'function_call': {'id': 'adk-445f1f9a-8a7f-452e-90ce-ff4f470fe770', 'args': {'request': 'economic theories related to technological unemployment and job creation'}, 'name': 'simple_search_agent'}}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 78, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 78}], 'prompt_token_count': 2902, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 2902}], 'total_token_count': 2980, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-fdb59cf2-4f91-4c88-8c07-

{'content': {'parts': [{'function_response': {'id': 'adk-445f1f9a-8a7f-452e-90ce-ff4f470fe770', 'name': 'simple_search_agent', 'response': {'result': 'Several economic theories attempt to explain the complex relationship between technological advancements, unemployment, and job creation. Here\'s an overview of some key perspectives:\n\n**1. Technological Unemployment Theory:**\n\n*   **Definition:** This theory posits that technological progress, automation, and mechanization lead to job losses because machines can perform tasks previously done by humans.\n*   **Structural Unemployment:** Technological unemployment is considered a type of structural unemployment, resulting from the restructuring of industries and reduced demand for labor in certain sectors.\n*   **Temporary vs. Permanent:** The unemployment can be temporary if workers can retrain and find new jobs. However, it can become permanent if skills become obsolete.\n\n**2. Compensation Theory:**\n\n*   **Main Idea:** This theo

{'content': {'parts': [{'text': 'Key economic theories include: Technological Unemployment Theory, Compensation Theory, Skills-Biased Technological Change (SBTC), Search-Matching Theory, and Demand-Oriented Theories (Keynesian). These theories offer different perspectives on how technology impacts job displacement and creation, wage inequality, and overall employment levels.\n'}], 'role': 'model'}, 'usage_metadata': {'candidates_token_count': 60, 'candidates_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 60}], 'prompt_token_count': 3825, 'prompt_tokens_details': [{'modality': <MediaModality.TEXT: 'TEXT'>, 'token_count': 3825}], 'total_token_count': 3885, 'traffic_type': <TrafficType.ON_DEMAND: 'ON_DEMAND'>}, 'invocation_id': 'e-fdb59cf2-4f91-4c88-8c07-dbdb25cd514c', 'author': 'research_agent', 'actions': {'state_delta': {}, 'artifact_delta': {}, 'requested_auth_configs': {}}, 'id': 'Jdmx3O4i', 'timestamp': 1748877682.067006}


{'content': {'parts': [{'text': '```markdown\n## AI Agents\' Impact on the Labor Market: A Research Report\n\nThis report summarizes research findings on the multifaceted impacts of AI Agents on the labor market, addressing key questions across foundational understanding, in-depth analysis, context/connections, practical implications, and future outlook.\n\n### I. Foundational Understanding\n\n*   **Definition of an AI Agent in the Labor Market:**\n    *   An AI agent is a sophisticated software program that **autonomously performs tasks**, **perceives its environment**, **makes decisions**, **takes action**, and **learns and improves** over time. In the labor market, AI agents automate tasks, boost productivity, create new job roles, improve decision-making, and reshape the workplace.\n\n*   **Types of AI Agents:**\n    *   Different types of AI Agents include:\n        *   **Simple Reflex Agents:** React to environmental conditions based on pre-defined rules (e.g., thermostat).\n    

In [40]:
from IPython.display import display, Markdown

display(Markdown(response_text))

Okay, I can help you create a research plan to deeply explore the impact of AI Agents on the labor market. Here's a structured plan with questions to guide your research:

*   **Foundational Understanding**
    *   What is the definition of an AI Agent in the context of the labor market?
    *   What are the different types of AI Agents currently being used or developed (e.g., chatbots, robotic process automation, virtual assistants)?
    *   What are the historical trends of automation and technological advancements and their impact on employment?
    *   What are the key economic theories related to technological unemployment and job creation?

*   **In-Depth Analysis**
    *   How are AI Agents currently being used in different industries and sectors?
    *   What specific tasks and roles are most susceptible to automation by AI Agents?
    *   What are the arguments for and against the displacement of human workers by AI Agents?
    *   What is the impact of AI Agents on the quality of jobs (e.g., wages, benefits, working conditions, skill requirements)?
    *   How is the introduction of AI Agents affecting the demand for different skill sets?
    *   What new job roles and opportunities are emerging as a result of AI Agent development and implementation?
    *   How are AI Agents changing the nature of work (e.g., remote work, gig economy, collaboration between humans and AI)?
    *   What are the psychological and social impacts on workers who are displaced or whose jobs are significantly altered by AI Agents?

*   **Context and Connections**
    *   How does the impact of AI Agents on the labor market differ across countries and regions, considering factors like economic development, education levels, and regulatory environments?
    *   What are the ethical considerations surrounding the use of AI Agents in the workplace (e.g., bias, fairness, transparency, accountability)?
    *   How are labor unions and worker advocacy groups responding to the increasing presence of AI Agents in the workplace?
    *   What are the legal and regulatory frameworks governing the use of AI Agents in employment, and how might these need to evolve?
    *   How do AI Agents affect income inequality and wealth distribution?
    *   How does the development and deployment of AI Agents relate to broader trends in globalization and technological change?

*   **Practical Implications**
    *   What strategies can businesses and organizations adopt to manage the transition to an AI-driven workforce effectively?
    *   What policies can governments implement to mitigate the negative impacts of AI Agents on employment and promote a more equitable distribution of benefits?
    *   What role should education and training programs play in preparing workers for the changing demands of the labor market?
    *   How can individuals adapt and develop the skills needed to remain competitive in an AI-driven economy?
    *   What are some examples of successful (or unsuccessful) implementations of AI Agents in the workplace, and what lessons can be learned from these cases?
    *   How can we measure and evaluate the overall impact of AI Agents on productivity, economic growth, and social well-being?

*   **Future Outlook**
    *   What are the emerging trends and future directions in AI Agent technology that are likely to further impact the labor market?
    *   What are the potential long-term consequences of widespread AI Agent adoption on the nature of work, the structure of the economy, and social inequality?
    *   What are the potential risks and challenges associated with the increasing reliance on AI Agents in the workplace, and how can these be addressed?
    *   What are the potential opportunities for creating a more inclusive and prosperous future through the responsible and ethical development and deployment of AI Agents?
    *   How can we ensure that the benefits of AI Agents are shared broadly and that the transition to an AI-driven economy is managed in a way that benefits all members of society?

This research plan should provide a comprehensive framework for exploring the multifaceted impact of AI Agents on the labor market. Remember to critically evaluate your sources and consider multiple perspectives to form your own informed opinion.
Okay, I will use the search tool to answer the questions in your research plan regarding the impact of AI Agents on the labor market. Here are the search results, organized by question:

**Foundational Understanding**

*   **What is the definition of an AI Agent in the context of the labor market?**
An AI agent in the labor market is a software program that autonomously performs tasks, perceives its environment, makes decisions, takes action, and learns/improves over time. They automate tasks, boost productivity, create new job roles, improve decision-making, and reshape the workplace.

*   **What are the different types of AI Agents currently being used or developed (e.g., chatbots, robotic process automation, virtual assistants)?**
Different types of AI Agents include: Simple Reflex Agents, Model-Based Reflex Agents, Goal-Based Agents, Utility-Based Agents, Learning Agents, Hierarchical Agents, and Multi-Agent Systems. They are used for automating routine tasks, customer service (chatbots), data analysis, and administrative tasks.

*   **What are the historical trends of automation and technological advancements and their impact on employment?**
Historically, automation has led to both job displacement (especially in routine tasks) and job creation (new jobs requiring specialized skills). It has also contributed to wage stagnation/decline for some workers and increased labor market inequality.

*   **What are the key economic theories related to technological unemployment and job creation?**
Key economic theories include: Technological Unemployment Theory, Compensation Theory, Skills-Biased Technological Change (SBTC), Search-Matching Theory, and Demand-Oriented Theories (Keynesian). These theories offer different perspectives on how technology impacts job displacement and creation, wage inequality, and overall employment levels.
```markdown
## AI Agents' Impact on the Labor Market: A Research Report

This report summarizes research findings on the multifaceted impacts of AI Agents on the labor market, addressing key questions across foundational understanding, in-depth analysis, context/connections, practical implications, and future outlook.

### I. Foundational Understanding

*   **Definition of an AI Agent in the Labor Market:**
    *   An AI agent is a sophisticated software program that **autonomously performs tasks**, **perceives its environment**, **makes decisions**, **takes action**, and **learns and improves** over time. In the labor market, AI agents automate tasks, boost productivity, create new job roles, improve decision-making, and reshape the workplace.

*   **Types of AI Agents:**
    *   Different types of AI Agents include:
        *   **Simple Reflex Agents:** React to environmental conditions based on pre-defined rules (e.g., thermostat).
        *   **Model-Based Reflex Agents:** Use an internal model of the environment to make decisions.
        *   **Goal-Based Agents:** Make decisions based on achieving specific goals.
        *   **Utility-Based Agents:** Consider both goals and the "utility" of different outcomes.
        *   **Learning Agents:** Adapt and improve performance over time.
        *   **Hierarchical Agents:** Break down complex problems into smaller tasks.
        *   **Multi-Agent Systems:** Collaborate or compete to achieve objectives.
    *   These agents are used for automating routine tasks, customer service (chatbots), data analysis, and administrative tasks.

*   **Historical Trends of Automation and Technological Advancements:**
    *   Historically, automation has led to both **job displacement** (especially in routine tasks) and **job creation** (new jobs requiring specialized skills).  It has also contributed to **wage stagnation/decline** for some workers and **increased labor market inequality**.

*   **Key Economic Theories:**
    *   Key economic theories related to technological unemployment and job creation include:
        *   **Technological Unemployment Theory:** Technology leads to job losses.
        *   **Compensation Theory:** The economy can compensate for job displacement through new jobs and increased demand.
        *   **Skills-Biased Technological Change (SBTC):** Technology increases demand for high-skilled workers and reduces demand for low-skilled workers.
        *   **Search-Matching Theory:** Unemployment results from frictions in the labor market.
        *   **Demand-Oriented Theories (Keynesian):** Government intervention can stimulate demand and create jobs.
    *   These theories offer different perspectives on how technology impacts job displacement and creation, wage inequality, and overall employment levels.

### II. In-Depth Analysis (To be populated with further research)

*   How are AI Agents currently being used in different industries and sectors?
*   What specific tasks and roles are most susceptible to automation by AI Agents?
*   What are the arguments for and against the displacement of human workers by AI Agents?
*   What is the impact of AI Agents on the quality of jobs?
*   How is the introduction of AI Agents affecting the demand for different skill sets?
*   What new job roles and opportunities are emerging?
*   How are AI Agents changing the nature of work?
*   What are the psychological and social impacts on workers?

### III. Context and Connections (To be populated with further research)

*   How does the impact of AI Agents differ across countries and regions?
*   What are the ethical considerations surrounding the use of AI Agents?
*   How are labor unions and worker advocacy groups responding?
*   What are the legal and regulatory frameworks governing the use of AI Agents?
*   How do AI Agents affect income inequality and wealth distribution?
*   How does the development of AI Agents relate to broader trends in globalization?

### IV. Practical Implications (To be populated with further research)

*   What strategies can businesses and organizations adopt?
*   What policies can governments implement?
*   What role should education and training programs play?
*   How can individuals adapt and develop the skills needed?
*   What are some examples of successful (or unsuccessful) implementations?
*   How can we measure and evaluate the overall impact of AI Agents?

### V. Future Outlook (To be populated with further research)

*   What are the emerging trends and future directions in AI Agent technology?
*   What are the potential long-term consequences of widespread AI Agent adoption?
*   What are the potential risks and challenges?
*   What are the potential opportunities for creating a more inclusive future?
*   How can we ensure that the benefits of AI Agents are shared broadly?

This report provides a starting point for understanding the complex impacts of AI Agents on the labor market. Further research is needed to address the remaining questions and develop a more comprehensive understanding.
```

## Bonus: Finish the remaining researches

Note that the above result has many open questions not yet researched. Try to build an agent tree that completes the research. Some possible directions:

- Use LoopAgent, refer to the QuickStart notebook of how to exit_loop.
- Optimize the prompt of multiple_search agent so it conducts more thorough searches.

## Bonus: Format with grounding data

The simple_search_agent returns not only the search results summary, but also the grounding web pages and references / citations.